In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from config import api_key
from config import gkey
import gmaps
import os
import json

In [2]:
weather_df = pd.read_csv("cities.csv")
weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,date
0,Cherskiy,100,RU,1611861133,85,68.7500,161.3000,-26.90,2.80,2021-01-28 19:12:13
1,Raudeberg,100,NO,1611861133,76,61.9875,5.1352,33.01,21.23,2021-01-28 19:12:13
2,São José da Coroa Grande,22,BR,1611861133,71,-8.8978,-35.1478,81.45,10.33,2021-01-28 19:12:13
3,Mataura,92,NZ,1611860640,81,-46.1927,168.8643,48.00,7.00,2021-01-28 19:04:00
4,Hilo,90,US,1611860710,78,19.7297,-155.0900,69.80,9.22,2021-01-28 19:05:10
...,...,...,...,...,...,...,...,...,...,...
609,Marsh Harbour,49,BS,1611872808,70,26.5412,-77.0636,66.02,26.57,2021-01-28 22:26:48
610,Arauca,4,CO,1611872809,44,7.0847,-70.7591,87.82,5.70,2021-01-28 22:26:49
611,Biri,86,PH,1611872809,81,12.6833,124.3636,78.28,22.48,2021-01-28 22:26:49
612,Mbandaka,100,CD,1611872809,78,0.0487,18.2603,73.54,3.51,2021-01-28 22:26:49


In [15]:
gmaps.configure(api_key=gkey)

# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"]

In [10]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=3)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
type(locations)

pandas.core.frame.DataFrame

In [12]:
city_df = weather_df.loc[(weather_df["Max Temp"] < 80) \
                                & (weather_df["Max Temp"] > 70) \
                                & (weather_df["Wind Speed"] < 10) \
                                & (weather_df["Cloudiness"] == 0)].dropna() 

In [17]:
city_df['Hotel'] = ""
city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,date,Hotel
29,Lima,0,PE,1611860858,57,-12.0432,-77.0282,78.80,9.22,2021-01-28 19:07:38,
92,Araouane,0,ML,1611872676,18,18.9048,-3.5265,70.77,9.33,2021-01-28 22:24:36,
96,Caravelas,0,BR,1611872677,81,-17.7125,-39.2481,77.09,6.20,2021-01-28 22:24:37,
102,Port Hedland,0,AU,1611872678,94,-20.3167,118.5667,78.80,6.91,2021-01-28 22:24:38,
110,Saint-Philippe,0,RE,1611872681,65,-21.3585,55.7679,78.80,6.91,2021-01-28 22:24:41,
130,São Filipe,0,CV,1611872686,67,14.8961,-24.4956,73.87,4.63,2021-01-28 22:24:46,
164,Nouadhibou,0,MR,1611872641,53,20.9310,-17.0347,71.60,9.22,2021-01-28 22:24:01,
223,Monrovia,0,LR,1611872596,94,6.3005,-10.7969,77.00,8.52,2021-01-28 22:23:16,
245,Guerrero Negro,0,MX,1611872716,30,27.9769,-114.0611,70.30,9.78,2021-01-28 22:25:16,
267,Moa,0,CU,1611872721,65,20.6569,-74.9403,77.25,9.40,2021-01-28 22:25:21,


In [ ]:
target_city = "Bothel, Washington"

# Build the endpoint URL
target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(target_city, gkey)
target_url